In [1]:
import pandas as pd
#import os
#pip install openpyxl
import numpy as np
#import scipy as sp
import math 

In [21]:
Data=pd.read_excel("DataCS-2018-2021.xlsx",
                   sheet_name=None,
                   usecols="B:P",
                   skipfooter=15,
                   header=7)



#LOADING SHEETS
Data18=Data['2018']
Data19=Data['2019']
Data20=Data['2020']
Data21=Data['2021']

#Data.keys()
#RENAME

# RENAME COLUMNS
new_columns = {0: 'Ma CK', 
               1: 'Ten Cong ty',
               2: 'San',
               3: 'Nhom nganh',
               4: 'Tong Tai san',
               5: 'Tong No phai tra',
               6: 'Von Chu so huu',
               7: 'No ngan han',
               8: 'No dai han',
               9: 'Tai san co dinh',
               10: 'Gia von hang ban',
               11: 'Loi nhuan truoc thue',
               12: 'Chi phi lai vay',
               13: 'Von hoa thi truong',
               14: 'ROA'               
              }

#RENAME and DROP ROWS

Data18=Data18.rename(columns={Data18.columns[i]: new_col for i, new_col in new_columns.items()})
Data19=Data19.rename(columns={Data19.columns[i]: new_col for i, new_col in new_columns.items()})
Data20=Data20.rename(columns={Data20.columns[i]: new_col for i, new_col in new_columns.items()})
Data21=Data21.rename(columns={Data21.columns[i]: new_col for i, new_col in new_columns.items()})

#DROP ROW 1-2
Data18.drop([0,1], axis=0, inplace=True)
Data19.drop([0,1], axis=0, inplace=True)
Data20.drop([0,1], axis=0, inplace=True)
Data21.drop([0,1], axis=0, inplace=True)

# LOC trong San Giao dich HOSE HNX UPCOM
San_GD = ['HOSE','HNX','UPCOM']
Data18 = Data18[Data18['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data19 = Data19[Data19['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data20 = Data20[Data20['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)
Data21 = Data21[Data21['San'].isin(San_GD)].sort_values(by=['Von hoa thi truong'], ascending=False)



In [22]:
# Tong hop Du lieu tu Nam 2018 - 2021
Data18['Nam']=2018
Data19['Nam']=2019
Data20['Nam']=2020
Data21['Nam']=2021
DataAll=pd.concat([Data18, Data19,Data20,Data21]).reset_index(drop=True)



In [23]:
#Don Vi la Unit: Million M
DataAll.iloc[:,4:13]=DataAll.iloc[:,4:13]//10**6 # Chia lay lam tron
#Von hoa thi truong, Unit: Ty dong, Billion  
DataAll['Von hoa thi truong']=DataAll['Von hoa thi truong'].apply(np.ceil)  #lam trong so tu nhien


In [24]:
# FILTER : TONG TAI SAN > 100B; Von Hoa thi truong > 100B
DataAll=DataAll[(DataAll['Tong Tai san'] >= 100*10**3) & (DataAll['Von hoa thi truong'] >= 100 )].reset_index(drop=True)


In [40]:
# Tinh toan Nang cao
# SIZE = natural logarithym LN(Total ASSETS)
DataAll['SIZE'] = np.log(DataAll['Tong Tai san'].astype('float'))
# EBIT = LNTT + Chi phi lai vay => PRO = EBIT/Total Assets ; RISK=Std(EBIT)
DataAll['EBIT']=DataAll['Loi nhuan truoc thue']+abs(DataAll['Chi phi lai vay'])
# Profitability PRO = EBIT/Tong Tai san
DataAll['PROF']=DataAll['EBIT']/DataAll['Tong Tai san']
# LIQD	Tổng tài sản ngắn hạn/ Nợ ngắn hạn
DataAll['LIQD']=DataAll['Tong Tai san']/DataAll['No ngan han']
# UNIQ	Giá vốn hàng bán/Tổng tài sản
DataAll['UNIQ']=DataAll['Gia von hang ban']/DataAll['Tong Tai san']
# Tính hữu hình	TANG	Tài sản cố định/ Tổng tài sản
DataAll['TANG']=DataAll['Tai san co dinh']/DataAll['Tong Tai san']

In [54]:
DataAll.iloc[:,[0,1,15,16,18,19,20,21]]

,Ma CK,Ten Cong ty,Nam,SIZE,PROF,LIQD,UNIQ,TANG
0,ACV,Cảng Hàng không VN,2018,17.797523,0.143837,6.978783,-0.154697,0.344006
1,MSN,Tập đoàn Masan,2018,17.983394,0.137992,4.088415,-0.407352,0.45222
2,VNM,VINAMILK,2018,17.436275,0.323905,3.511987,-0.748019,0.357687
3,HPG,Hòa Phát,2018,18.175074,0.13565,3.455668,-0.564612,0.163412
4,SAB,SABECO,2018,16.923086,0.242578,3.774534,-1.245797,0.204191
...,...,...,...,...,...,...,...,...
3003,SGD,Sách GD TP.HCM,2021,11.734556,0.034716,2.503329,-1.068984,0.207169
3004,HEJ,Tư vấn Xây dựng Thủy Lợi VN,2021,12.127484,0.040555,1.774199,-0.560898,0.090738
3005,BT1,Bảo vệ thực vật 1 TW,2021,12.419715,0.055456,1.722002,-0.834309,0.149853
3006,SPV,Thủy đặc sản,2021,12.780783,0.067857,1.847725,-1.025228,0.349431
